<a href="https://colab.research.google.com/github/PhoebeLu1011/1132code/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/13pxMrVg5P5oycggaXig4J9LC2q3G7aSkEwA5ktoAmqI/edit?usp=sharing').sheet1

In [ ]:
# 讀取所有數據
rows = gsheets.get_all_records()

import pandas as pd
df = pd.DataFrame(rows)

In [ ]:
df.head()

,Patient_ID,Symptom_Description
0,P004,最近行走時膝蓋疼痛加劇，希望能加強復健運動。
1,P009,左腳腫脹，有輕微疼痛感。
2,P008,腰部痠痛影響翻身與起床。
3,P004,偶爾感到心悸與冒冷汗。
4,P003,呼吸訓練時覺得胸口緊緊的。


In [ ]:
df.describe()

,Patient_ID,Symptom_Description
count,30,30
unique,15,14
top,P004,偶爾感到心悸與冒冷汗。
freq,4,5


In [ ]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 48.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=42d9a7700657ad563e0259676794a77606133f94fb6684977a3f87e4dfebb98b
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [ ]:
import jieba
from snownlp import SnowNLP
from collections import Counter
import re
import plotly.express as px

In [ ]:
for _, row in df.iterrows():
    word = row['Symptom_Description']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.301 seconds.
DEBUG:jieba:Loading model cost 1.301 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [ ]:
df.head()

,Patient_ID,Symptom_Description
0,P004,最近行走時膝蓋疼痛加劇，希望能加強復健運動。
1,P009,左腳腫脹，有輕微疼痛感。
2,P008,腰部痠痛影響翻身與起床。
3,P004,偶爾感到心悸與冒冷汗。
4,P003,呼吸訓練時覺得胸口緊緊的。


In [ ]:
def clean_text(text):
    # 移除特殊符號
    text = re.sub(r'[!@#$%^&*()_+\-=\[\]{};\'\\:"|.<>/?~><？！~~...]', '', text)
    # 移除多餘空格
    text = re.sub(r'\s+', '', text)
    return text

In [ ]:
df['Cleaned_Description'] = df['Symptom_Description'].apply(clean_text)

# 顯示處理後的資料
df[['Symptom_Description','Cleaned_Description']].head()

# 中文斷詞
df['Tokenized'] = df['Cleaned_Description'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]


<ipython-input-17-b31a023f16df>:3: FutureWarning: Possible set symmetric difference at position 40
  text = re.sub(r'[!@#$%^&*()_+\-=\[\]{};\'\\:"|.<>/?~><？！~~...]', '', text)


In [ ]:
df['Cleaned_Description']

,Cleaned_Description
0,最近行走時膝蓋疼痛加劇，希望能加強復健運動。
1,左腳腫脹，有輕微疼痛感。
2,腰部痠痛影響翻身與起床。
3,偶爾感到心悸與冒冷汗。
4,呼吸訓練時覺得胸口緊緊的。
5,晚上常常失眠，無法入睡。
6,站立時會感到頭暈，血壓偏低。
7,左腳腫脹，有輕微疼痛感。
8,膝關節酸痛影響上下樓梯。
9,偶爾感到心悸與冒冷汗。


In [ ]:
df['Tokenized']

,Tokenized
0,"[最近, 行走, 時膝, 蓋, 疼痛, 加劇, ，, 希望, 能加強, 復, 健運動, 。]"
1,"[左腳, 腫脹, ，, 有輕微, 疼痛感, 。]"
2,"[腰部, 痠痛, 影響, 翻身, 與, 起床, 。]"
3,"[偶爾, 感到, 心悸, 與, 冒, 冷汗, 。]"
4,"[呼吸, 訓練, 時覺, 得, 胸口, 緊緊, 的, 。]"
5,"[晚上, 常常, 失眠, ，, 無法, 入睡, 。]"
6,"[站立, 時會, 感到, 頭暈, ，, 血壓, 偏低, 。]"
7,"[左腳, 腫脹, ，, 有輕微, 疼痛感, 。]"
8,"[膝, 關節, 酸痛, 影響, 上下, 樓梯, 。]"
9,"[偶爾, 感到, 心悸, 與, 冒, 冷汗, 。]"


In [ ]:
word_counts = Counter(all_words)
word_counts

Counter({'最近': 2,
         '行走': 2,
         '時膝': 2,
         '疼痛': 2,
         '加劇': 2,
         '希望': 2,
         '能加強': 2,
         '健運動': 2,
         '左腳': 4,
         '腫脹': 4,
         '有輕微': 4,
         '疼痛感': 4,
         '腰部': 1,
         '痠痛': 1,
         '影響': 5,
         '翻身': 1,
         '起床': 2,
         '偶爾': 5,
         '感到': 8,
         '心悸': 5,
         '冷汗': 5,
         '呼吸': 3,
         '訓練': 3,
         '時覺': 3,
         '胸口': 3,
         '緊緊': 3,
         '晚上': 2,
         '常常': 2,
         '失眠': 2,
         '無法': 2,
         '入睡': 2,
         '站立': 3,
         '時會': 3,
         '頭暈': 3,
         '血壓': 3,
         '偏低': 3,
         '關節': 3,
         '酸痛': 3,
         '上下': 3,
         '樓梯': 3,
         '行步': 1,
         '態訓練': 1,
         '時腳踝': 1,
         '不穩': 1,
         '肩膀': 1,
         '活動': 1,
         '範圍': 1,
         '容易': 1,
         '疲倦': 1,
         '無法長': 1,
         '時間': 1,
         '專注': 1,
         '咳嗽': 1,
         '持續': 1,
         '超過': 1,
   

In [ ]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['，', '希望','最近', '時會', '感到', '與', '出現', '晚上', '常常', '也', '在', '和', '時覺','可以' , '偶爾'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

In [ ]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [ ]:

print("圖表說明")
explanation = {
    "元素": ["橫軸（X 軸）", "縱軸（Y 軸）", "長條高度", "數字標籤"],
    "意義": [
        "出現最多次的關鍵字",
        "每個關鍵字在所有描述中出現的次數",
        "代表該詞出現頻率，高代表常被提到",
        "每個詞實際出現的次數（長條上方）"
    ]
}

df_exp = pd.DataFrame(explanation)
df_exp


圖表說明


,元素,意義
0,橫軸（X 軸）,出現最多次的關鍵字
1,縱軸（Y 軸）,每個關鍵字在所有描述中出現的次數
2,長條高度,代表該詞出現頻率，高代表常被提到
3,數字標籤,每個詞實際出現的次數（長條上方）


圖表分析與解釋: \
1. 「心悸」、「冷汗」等詞彙，出現了5次，表示這些症狀在描述中被頻繁提到，可能是許多人都有的症狀。
2. 「腫脹」、「疼痛感」、「左腳」等詞彙總共出現各4次，可能與疾病帶來的不適有關。
3. 「呼吸」、「站立」、「酸痛」、「胸口」、「關節」、「血壓」等詞彙總共出現各三次，可能與其中一些疾病所帶來的症狀發生位置或現象有關。
4. 「過敏」、「紅疹」、「皮膚」等詞會出現各兩次，可能是與某些特定疾病的病患才會發生的症狀。

In [ ]:
word_freq_df

,Word,Frequency
0,心悸,5
1,影響,5
2,冷汗,5
3,腫脹,4
4,有輕微,4
5,疼痛感,4
6,左腳,4
7,呼吸,3
8,緊緊,3
9,頭暈,3


In [ ]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-wqy-zenhei is already the newest version (0.9.45-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [ ]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

圖表解釋與分析:\
此文字雲圖呈現了 30 筆病患症狀描述中最常出現的關鍵字。字體越大代表該詞彙出現次數越多。從圖中可觀察到「心悸」、「腫脹」、「疼痛感」、「影響」等詞彙較為突出，顯示這些症狀在多數病患的陳述中頻繁出現，可能已對其日常生活造成影響。透過文字雲圖的視覺化呈現，不僅能直觀掌握主要病症類型，也反映出部分病患的情緒感受與生活困擾，為後續醫護介入與溝通提供了參考依據。